In [1]:
from pytket.circuit import Circuit, CircBox
from pytket.circuit.display import render_circuit_jupyter as draw


circ = Circuit(4).CX(0, 1).CX(1, 2).CX(0, 3).H(0, opgroup="H").CX(1, 0).H(2, opgroup="H").H(3, opgroup="H")

![](images/gadgetisation.png)

In [2]:
draw(circ)

In [3]:
new_circ = Circuit(2, name="H-Gadget").H(1).CZ(0, 1).SWAP(0, 1)

In [4]:
draw(new_circ)

In [5]:
H_gadget = CircBox(new_circ)

In [6]:
from pytket.passes import CustomPass
from pytket import Qubit, OpType


def gadgetise_hadamards(circ: Circuit) -> Circuit:
    h_count = circ.n_gates_of_type(OpType.H)

    circ_prime = Circuit(circ.n_qubits)
    z_ancillas = circ_prime.add_q_register("z_ancillas", h_count)
    ancilla_bits = circ_prime.add_c_register("bits", h_count)

    ancilla_index = 0

    for cmd in circ:
        if cmd.op.type != OpType.H:
            circ_prime.add_gate(cmd.op.type, cmd.args)
        else:
            circ_prime.add_gate(H_gadget, [cmd.qubits[0], z_ancillas[ancilla_index]])
            circ_prime.Measure(z_ancillas[ancilla_index], ancilla_bits[ancilla_index])
            circ_prime.X(cmd.qubits[0], condition_bits=[ancilla_bits[ancilla_index]], condition_value=1)
            ancilla_index += 1
            
    return circ_prime

In [7]:
replace_hadamards = CustomPass(gadgetise_hadamards)

In [8]:
replace_hadamards.apply(circ)

True

In [9]:
draw(circ)